In [1]:
import pandas as pd
import numpy as np

from common.serialization import pickle_save, pickle_load

from feature_pr import * #make_counters, make_counters_test, compress_vals, replace_val, make_pairs

In [2]:
data = pd.read_csv('data/credit_train.csv', sep=';', encoding='cp1251')
data_test = pd.read_csv('data/credit_test.csv', sep=';', encoding='cp1251')

In [3]:
a = ['РЕСП', 'ОБЛ', 'ОБЛАСТЬ', 'КРАЙ', 'ОБЛ.', 'РЕСП.', 'Р-Н', 'АО', 'КРАЙ.', '-', 'Г', 'Г.', 'АОБЛ', 'РЕСПУБЛИКА',
    'ОКРУГ', 'АВТОНОМНЫЙ']

data.monthly_income.fillna(data.monthly_income.median(), inplace=True)
data.living_region.fillna('Н', inplace=True)
data.credit_count.fillna(-1, inplace=True)
data.overdue_credit_count.fillna(-1, inplace=True)
data['credit_sum'] = data.credit_sum.apply(lambda x: float(x.replace(',', '.')))
data['score_shk'] = data.score_shk.apply(lambda x: float(x.replace(',', '.')))
data['living_region'] = data.living_region.apply(lambda x: ''.join([y for y in x.split() if y not in a]))

data_test.monthly_income.fillna(data.monthly_income.median(), inplace=True)
data_test.living_region.fillna('Н', inplace=True)
data_test.credit_count.fillna(-1, inplace=True)
data_test.overdue_credit_count.fillna(-1, inplace=True)
data_test['credit_sum'] = data_test.credit_sum.apply(lambda x: float(x.replace(',', '.')))
data_test['score_shk'] = data_test.score_shk.apply(lambda x: float(x.replace(',', '.')))
data_test['living_region'] = data_test.living_region.apply(lambda x: ''.join([y for y in x.split() if y not in a]))


c = data_test.living_region.unique()
d = data.living_region.unique()
for elem in c:
    if elem not in d:
        data_test.loc[data_test.living_region == elem, 'living_region'] = 'Н'

In [4]:
counters_cols = ['gender', 'marital_status', 'job_position', 'credit_month', 'tariff_id', 
                'education', 'living_region', 'age', 'credit_count', 'overdue_credit_count', 'age_group']
other_cols = ['credit_sum', 'score_shk', 'monthly_income']

# Объединим редко встречающиеся значения признаков
data.loc[data.credit_count > 7, 'credit_count'] = 7
data_test.loc[data_test.credit_count > 7, 'credit_count'] = 7
data.loc[data.overdue_credit_count > 2, 'credit_count'] = 2
data_test.loc[data_test.credit_count > 2, 'credit_count'] = 2

compress_vals(data, data_test, 'living_region', 30, 'Н')
compress_vals(data, data_test, 'tariff_id', 30, 2.0)
compress_vals(data, data_test, 'job_position', 30, 'N')

data['age_group'] = 0
data_test['age_group'] = 0
a = [range(18, 22), range(22, 25), range(25, 38), range(38, 50), range(50, 58), range(58, 66), range(66, 100)]
for i in range(len(a)):
    for val in a[i]:
        data.loc[data.age == val, 'age_group'] = i
        data_test.loc[data_test.age == val, 'age_group'] = i

replace_val(data, 'credit_month', [30, 31, 32], 36)
replace_val(data_test, 'credit_month', [30, 31, 32], 36)
replace_val(data, 'credit_month', [21, 22, 23, 25, 26, 27, 28, 29], 24)
replace_val(data_test, 'credit_month', [21, 22, 23, 25, 26, 27, 28, 29], 24)
replace_val(data, 'credit_month', [17], 18)
replace_val(data_test, 'credit_month', [17], 18)

#пары признаков
pairs_to_col = ['gender', 'marital_status', 'credit_count', 'overdue_credit_count', 'age_group', 'job_position',
    'education']

data, pair_cols = make_pairs(data, pairs_to_col)
data_test, pair_cols = make_pairs(data_test, pairs_to_col)

for col in pair_cols:
    compress_vals(data, data_test, col, 30, 'N')

In [5]:
#счетчики
X_train = np.hstack((make_counters(X=data[counters_cols + pair_cols].values, y=data.open_account_flg.values, 
                                   n_folds=20), data[other_cols]))
X_test = np.hstack((make_counters_test(data_test[counters_cols + pair_cols].values, data[counters_cols + pair_cols].values, 
                                       data.open_account_flg.values), data_test[other_cols]))

In [7]:
pickle_save(X_train, 'data/X_train.pkl')
pickle_save(X_test, 'data/X_test.pkl')
pickle_save(data_test.client_id.values, 'data/_ID_.pkl')
pickle_save(data.open_account_flg.values, 'data/y_train.pkl')